# Natural Language Processing HW-2 Report

In [1]:
#Author: Wei-Cheng Chen
#Student ID: 0816065
#HW ID: hw2
#Due Date: 04/15/2022

In [2]:
import pandas as pd
import nltk
import spacy
from spacy import displacy


nlp = spacy.load("en_core_web_sm")

In [7]:
df = pd.read_csv("data.csv")
df.sample(10)

,id,S,V,O,sentence
2098,250,the defending champion Philadelphia Eagles,handily in,the league championship,They defeated the defending champion Philadelp...
532,635,it,may be,a sign,NYT News Service And when `` Presidio Med '' f...
1733,1350,Baker Hughes Inc.,releases its weekly survey of,the number of active oil and gas drilling rigs,Houston : Baker Hughes Inc. releases its weekl...
1588,1202,him of manipulation,to protect hold on,power,He said keeping her off the ballot was part of...
947,2258,you,find,the ocean conditions,"Even if you just want to take a dip , you 'll ..."
1753,2124,the products,flops as,Pecoriello,As for labeling the products flops as Pecoriel...
788,1227,his smile,had receded and,his tone,"But within seconds , his smile had receded and..."
2414,333,It,has could swell,its roster of shares,It has outstanding stock options that could sw...
2408,1914,outstanding stock options,that could swell,15 percent,It has outstanding stock options that could sw...
988,1934,it,has kept,the UPS strike,"Meanwhile , Federal Express said it has kept a..."


## 方案一
  
  使用 noun_chunks 查看是否符合 S 或 O  
  並確認他的 head 是不是符合動詞 V 

In [4]:
out = pd.DataFrame(columns=['id','label'])
out['id'] = df['id']

for i in df.itertuples() :
        
    checks = False
    checko = False

    for noun in nlp(i.sentence).noun_chunks :
        if noun.text in i.S and noun.root.head.text in i.V :
            checks = True
        elif noun.text in i.O and noun.root.head.text in i.V :
            checko = True
    
    out.at[i.Index, 'label'] = int(checks and checko)

out.to_csv("./outputs_v1.csv",index = False)

## 方案二
  
  使用 noun_chunks 查看是否符合 S 或 O  
  並確認他的 head 是不是符合動詞 V  
  另外檢查 S 中是否為 subj 組成  
  且 O 中是否為 obj 或 attr 組成  

In [5]:
out = pd.DataFrame(columns=['id','label'])
out['id'] = df['id']

for i in df.itertuples() :
        
    checks = False
    checko = False

    for noun in nlp(i.sentence).noun_chunks :
        if noun.text in i.S and noun.root.head.text in i.V and noun.root.dep_.find("subj") > 0 :
            checks = True
        elif noun.text in i.O and noun.root.head.text in i.V and ( noun.root.dep_.endswith('obj') or noun.root.dep_.endswith('attr') ):
            checko = True
    
    out.at[i.Index, 'label'] = int(checks and checko)

out.to_csv("./outputs_v2.csv",index = False)

## 方案三
  
  使用 subtree 查看 動詞的前後 是否符合 S 跟 O  
  並確認他的 左樹是否符合S的條件 右樹是否符合O的條件  
  另外用 ent 檢查 名詞是否被切割 

In [6]:
out = pd.DataFrame(columns=['id','label'])
out['id'] = df['id']

for i in df.itertuples() :
    check = False
    sent = nlp(i.sentence)
    ent = list(sent.ents)
    nch = list(sent.noun_chunks)
    Verb = [ x.text for x in sent if (x.pos_ == "VERB" or x.pos_ == "AUX") ]
    
    for t in sent :
        if (t.pos_ == "VERB" or t.pos_ == "AUX") and (t.head.text == t.text or t.head.text not in Verb) :
            L = list(t.lefts)
            R = list(t.rights)
            
            checks = False
            checko = False
            
            for l in L :
                if l.pos_ != "NOUN" :
                        continue
                line = "".join([ x.text_with_ws for x in [ w for w in list(l.subtree) ] ]).strip()
                if line.find(i.S) != -1 :
                    checks = True
            
            for r in R :
                if r.text in Verb :
                    continue
                q = [ x.text for x in ent if x.text.find(r.text) > -1 ]
                line = "".join([ x.text_with_ws for x in [ w for w in list(r.subtree) ] ]).strip()
                if len(q) > 0 and line.find(q[0]) == -1 and q[0].find(" ") != -1 :
                    line = q[0]
                if line.find(i.O) != -1 :
                    checko = True 
            
            if checks and checko :
                check =True
                break
    
    out.at[i.Index, 'label'] = int(check)

out.to_csv("./outputs_v3.csv",index = False)